# Machine Learning (revenue_prediction)

In [52]:
# Packages / libraries
import os
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
import json
import datetime 

## Loading the Raw Data

In [15]:
# Loading the data

with open(r"C:\Users\user\Downloads\accounton_data.json\accounton_data.json",'r') as f:
    raw_data = json.loads(f.read())

In [41]:
# Flatten data
data = pd.json_normalize(raw_data)

# Data Preprocessing

In [ ]:
# convert the CreationDate column into DateTime type Delet the companies the created after 2019 because it had many NaN values
data['creation_date'] = pd.to_datetime(data['creation_date']).datetime.to_period('Y')

##  Checking for NULL Values

In [20]:
# Checking for null values
data.isnull().sum()

company_name            0
nace_code               0
vat_number              0
zipcode                 0
city                    0
                    ...  
staff_costs.2019    31132
staff_costs.2018    31569
staff_costs.2016    32485
staff_costs.2017    33830
staff_costs.2015    23800
Length: 75, dtype: int64

In [54]:
df1 = data[data['ebit.2019'].notna()]
# df1

## Checking for the duplicated values

In [29]:
# checking the duplicated company
bool_series = data.duplicated()
bool_series

0        False
1        False
2        False
3        False
4        False
         ...  
49999    False
50000    False
50001    False
50002    False
50003    False
Length: 50004, dtype: bool